In [118]:
import math
from datetime import date
import numpy as np

'''base class with standard market inputs'''
class FXOptionNumerical(object): 
    
    #def __init__(self, CallPutFlag, S, K, sigma,  rFor, rDom, dateEnd, N, params, notional, premConvention):
    def __init__(self, S, K,   rFor, rDom, dateEnd, N, params, notional, premConvention):
      
        #date format: date(2017, 6, 30)
        
        #constructors 
        
        #self.CallPutFlag = CallPutFlag #pass it as dictionary element below
        self.S = S
        self.K = K
        #self.sigma = sigma  #pass it as dictionary element below
        self.rFor = rFor
        self.rDom = rDom
        
        ###-------------------------------------
        # Convert date to unit of years
        ###--------------------------------
  
        self.dateEnd = dateEnd
        today = date.today()
        self.Tau = (self.dateEnd - today).days / 365.0

        if self.Tau == 0:
            raise ValueError('Option Expired')
            
        print('Time is :'+str(self.Tau)) #sanity check
        
        self.N = max(1, N)  #timesteps
        self.STs = None
        
        self.notional = notional
        self.premConvention = premConvention
        #derived params
        
        self.pu = params.get('pu', 0)
        self.pd = params.get('pd', 0)
        self.sigma = params.get('sigma', 0.001)
        self.is_call = params.get('is_call', True)
        self.is_european = params.get('is_eu', True)
    
        self.dt = self.Tau / float(N)
        self.df = math.exp(-(rDom - rFor) * self.dt)

###===============================================     
#implementation of tree inheriting from base class
###================================================

class BinomialTree(FXOptionNumerical):
    
    def _setup_parameters_(self):
        
        #volatility adjusted path
        #self.u = 1 + self.pu
        self.u = math.exp(self.sigma * pow(self.dt,0.5))

        #self.d = 1 - self.pd
        self.d = math.exp(-self.sigma * pow(self.dt, 0.5))
        
        #risk neutra probabilities
        self.qu  = (math.exp((self.rDom - self.rFor) * self.dt) - self.d)/(self.u -self.d)
        #print(self.qu)
        self.qd = 1 - self.qu
    
    #forward propagation 
    def _initialize_underlying_tree_(self):
        
        #tree's starting point is today, setup as numpy array 
        self.STs = [np.array([self.S])]  
        
        for i in range(self.N):
            branch = self.STs[i]
            #at 2nd node have 2 prices: multiply 2 by pu, and last one by pd
            #at 3rd node have 3 prices: multiply 3 by pu, and last one by pd ..and so forth
            next_branch = np.concatenate((branch* self.u, [branch[-1] * self.d]))  #merge two arrays
            self.STs.append(next_branch) 
                    
    #check payoff at final leg
    def _initialize_payoffs_tree_(self):
        
        return np.maximum(0, (self.STs[self.N]-self.K) if self.is_call
            else (self.K-self.STs[self.N]))
    
    def __check_early_exercise__(self, payoffs, node):
    
        early_ex_payoff = (self.STs[node] - self.K) if self.is_call \
            else (self.K - self.STs[node])
        #print('node ' +str(node))
        #print(early_ex_payoff)
        return np.maximum(early_ex_payoff, payoffs )
        
    #backward induction 
    def _traverse_tree_(self, payoffs):
        
        for i in reversed(range(self.N)):
            # The payoffs from NOT exercising the option
            payoffs = (payoffs[:-1] * self.qu +
                           payoffs[1:] * self.qd) * self.df

            # Payoffs from exercising, for American options
            if not self.is_european:
                payoffs = self.__check_early_exercise__(payoffs, i)

        return payoffs
    
    def __begin_traverse__(self):
        payoffs = self._initialize_payoffs_tree_()
        return self._traverse_tree_(payoffs)
    
    def price(self):
        self._setup_parameters_()
        self._initialize_underlying_tree_()
        payoffs = self.__begin_traverse__()
        price = payoffs[0]*self.notional / self.S
        return price

In [122]:
fxo_european = BinomialTree(S = 106.60, 
                  K = 109,  
                  rFor = 0.01788, 
                  rDom = -0.00818, 
                  dateEnd = date(2018, 9, 28),
                  N = 1000, 
                  params = {'pu': 0, 'pd': 0, 'sigma' : 0.0815, 'is_call': True, 'is_eu': True},
                  notional = 10000000,
                  premConvention = '')
fxo_european.price()

Time is :0.5780821917808219


106080.3765590871

In [126]:
fxo_american = BinomialTree(S = 106.60, 
                  K = 109,  
                  rFor = 0.01788, 
                  rDom = -0.0082, 
                  dateEnd = date(2018, 9, 28),
                  N = 1000,
                  params = {'pu': 0, 'pd': 0, 'sigma' : 0.0815, 'is_call': True, 'is_eu': False},
                  notional = 10000000,
                  premConvention = '')
fxo_american.price()

Time is :0.5780821917808219


112593.41276087683

In [69]:
500000*1.88 / 6.5

144615.38461538462